In [1]:
import numpy as np
import matplotlib.pyplot as plt
import scipy
import pandas as pd
from sklearn.impute import SimpleImputer
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder

%matplotlib inline

Data import and union

In [3]:
data = pd.read_csv('/data/home/krazheva/transactions_without_text.csv')
data_new = data.loc[data['date_key'] > '2021-03-01']
data = data.drop(data.index[119833238:123351243])
data = data.drop(['client_bik', 'partner_bik', 'direction'], axis=1)
data.head()

,date_key,amount,client_inn,partner_inn
0,2018-03-01,41000.00,1279230,1
1,2018-03-01,482300.00,1224261,13022
2,2018-03-01,15.72,1549773,119556
3,2018-03-01,4217742.30,976651,976651
4,2018-03-01,11.77,1413113,1003953


In [4]:
clients_1 = pd.read_csv('/data/home/krazheva/data_clients.csv')
clients_2 = clients_1.drop(['Клиент_ДатаРег.', 'Клиент_АдресРегистрацииОКАТО', 'Представитель_АдресЖительстваОКАТО', 'Представитель_НачалоДействия', 'Представитель_ОкончаниеДействия'], axis=1)
clients_2.head()

,inn,Клиент_ДатаНачалаОтн,Клиент_ДатаЗавершОтн,Клиент_Сегмент,Клиент_НаличиеЗПпроекта,years
0,1384214,26.03.2021,NaN,МБ,NaN,0
1,286507,25.10.2013,02.07.2019,МКП,NaN,34
2,648912,26.03.2021,NaN,МБ,NaN,0
3,1436001,11.02.2005,NaN,КБ,Д,53
4,918092,26.03.2021,NaN,МКП,NaN,0


In [5]:
clients_2['Клиент_ДатаЗавершОтн'] = clients_2['Клиент_ДатаЗавершОтн'].fillna('03.08.2022')

In [6]:
clients_2['Клиент_ДатаЗавершОтн'] = pd.to_datetime(clients_2['Клиент_ДатаЗавершОтн'])
clients_2['Клиент_ДатаНачалаОтн'] = pd.to_datetime(clients_2['Клиент_ДатаНачалаОтн'])
clients_2['Длина_Отн_в_днях'] = (clients_2['Клиент_ДатаЗавершОтн'] - clients_2['Клиент_ДатаНачалаОтн']).dt.days
clients_2 = clients_2.drop(['Клиент_ДатаНачалаОтн', 'Клиент_ДатаЗавершОтн'], axis=1)
clients_2.head()

/tmp/ipykernel_2693454/1207411023.py:1: UserWarning: Parsing dates in DD/MM/YYYY format when dayfirst=False (the default) was specified. This may lead to inconsistently parsed dates! Specify a format to ensure consistent parsing.
  clients_2['Клиент_ДатаЗавершОтн'] = pd.to_datetime(clients_2['Клиент_ДатаЗавершОтн'])
/tmp/ipykernel_2693454/1207411023.py:2: UserWarning: Parsing dates in DD/MM/YYYY format when dayfirst=False (the default) was specified. This may lead to inconsistently parsed dates! Specify a format to ensure consistent parsing.
  clients_2['Клиент_ДатаНачалаОтн'] = pd.to_datetime(clients_2['Клиент_ДатаНачалаОтн'])


,inn,Клиент_Сегмент,Клиент_НаличиеЗПпроекта,years,Длина_Отн_в_днях
0,1384214,МБ,NaN,0,347.0
1,286507,МКП,NaN,34,1931.0
2,648912,МБ,NaN,0,347.0
3,1436001,КБ,Д,53,5970.0
4,918092,МКП,NaN,0,347.0


In [7]:
clients_3 = clients_2.dropna()
clients_3.head()

,inn,Клиент_Сегмент,Клиент_НаличиеЗПпроекта,years,Длина_Отн_в_днях
3,1436001,КБ,Д,53,5970.0
8,1546173,МКП,Д,37,3928.0
20,1445248,МКП,Н,31,2253.0
23,783169,КБ,Д,44,2603.0
36,1504739,МКП,Д,34,2345.0


In [8]:
column_trans = ColumnTransformer(
[('imp_col2', SimpleImputer(missing_values=0, strategy='median'), [3])],
remainder='passthrough')
clients_4 = pd.DataFrame(column_trans.fit_transform(clients_3)[:, [1,2,3,0,4]])
clients_4.columns=clients_3.columns
clients_4.index=clients_3.index
clients_4.head()

,inn,Клиент_Сегмент,Клиент_НаличиеЗПпроекта,years,Длина_Отн_в_днях
3,1436001,КБ,Д,53.0,5970.0
8,1546173,МКП,Д,37.0,3928.0
20,1445248,МКП,Н,31.0,2253.0
23,783169,КБ,Д,44.0,2603.0
36,1504739,МКП,Д,34.0,2345.0


In [9]:
one_hot_encoded_data = pd.get_dummies(clients_4, columns = ['Клиент_Сегмент'])
one_hot_encoded_data.head()

,inn,Клиент_НаличиеЗПпроекта,years,Длина_Отн_в_днях,Клиент_Сегмент_КБ,Клиент_Сегмент_МБ,Клиент_Сегмент_МКП,Клиент_Сегмент_СБ
3,1436001,Д,53.0,5970.0,1,0,0,0
8,1546173,Д,37.0,3928.0,0,0,1,0
20,1445248,Н,31.0,2253.0,0,0,1,0
23,783169,Д,44.0,2603.0,1,0,0,0
36,1504739,Д,34.0,2345.0,0,0,1,0


In [10]:
clients = pd.get_dummies(one_hot_encoded_data, columns = ['Клиент_НаличиеЗПпроекта'])
clients.head()

,inn,years,Длина_Отн_в_днях,Клиент_Сегмент_КБ,Клиент_Сегмент_МБ,Клиент_Сегмент_МКП,Клиент_Сегмент_СБ,Клиент_НаличиеЗПпроекта_Д,Клиент_НаличиеЗПпроекта_Н
3,1436001,53.0,5970.0,1,0,0,0,1,0
8,1546173,37.0,3928.0,0,0,1,0,1,0
20,1445248,31.0,2253.0,0,0,1,0,0,1
23,783169,44.0,2603.0,1,0,0,0,1,0
36,1504739,34.0,2345.0,0,0,1,0,1,0


In [11]:
fraud = pd.read_csv('/data/home/krazheva/data_fraud.csv')
fraud.head()

,inn,month_fraud
0,249847,2021-03-01
1,170435,2020-04-01
2,1499321,2019-05-01
3,533212,2021-03-01
4,1623507,2021-03-01


In [12]:
fraud_sort = fraud.sort_values(by=['month_fraud'], ascending=False)
fraud_sort.head()

,inn,month_fraud
0,249847,2021-03-01
120,397979,2021-03-01
680,1894908,2021-03-01
184,1409774,2021-03-01
187,1409646,2021-03-01


In [13]:
fraud_new = fraud.loc[fraud['month_fraud'] < '2018-03-01']
index1 = fraud_new.iloc[:,0]
index1 = (index1.index.tolist())
fraud_new = fraud.loc[fraud['month_fraud'] < '2018-03-01']
fraud = fraud.drop(index1)
fraud.head()

,inn,month_fraud
0,249847,2021-03-01
1,170435,2020-04-01
2,1499321,2019-05-01
3,533212,2021-03-01
4,1623507,2021-03-01


In [14]:
fraud = fraud.assign(Fraud_presence = 1)
fraud['inn']=fraud['inn'].astype(int)
fraud.head()

,inn,month_fraud,Fraud_presence
0,249847,2021-03-01,1
1,170435,2020-04-01,1
2,1499321,2019-05-01,1
3,533212,2021-03-01,1
4,1623507,2021-03-01,1


In [15]:
reject = pd.read_csv('/data/home/krazheva/data_reject.csv')
reject = reject.assign(Reject_presence = 1)
reject.head()

,inn,РеестрОтказов_ТипУчастника,РеестрОтказов_КодОтказаСтрокой,month_reject,Reject_presence
0,826229,ИП,Отказ от закл.дог.,2017-07-01,1
1,183819,ИП,Отказ от закл.дог.,2017-07-01,1
2,1447227,ИП,Отказ от закл.дог.,2017-07-01,1
3,346763,Юридическое лицо,Отказ от закл.дог.,2017-07-01,1
4,1622547,Юридическое лицо,Отказ от закл.дог.,2017-07-01,1


In [16]:
reject_new = reject.loc[reject['month_reject'] < '2018-03-01']
index2 = reject_new.iloc[:,0]
index2 = (index2.index.tolist())
reject = reject.drop(index2)
reject.head()

,inn,РеестрОтказов_ТипУчастника,РеестрОтказов_КодОтказаСтрокой,month_reject,Reject_presence
115,309291,Юридическое лицо,Отказ в проведении операции,2018-12-01,1
166,1753304,Юридическое лицо,Отказ в проведении операции,2019-01-01,1
197,1238773,ИП,Отказ от закл.дог.,2018-12-01,1
229,1209749,Юридическое лицо,Отказ в проведении операции,2018-12-01,1
248,327939,Юридическое лицо,Отказ от закл.дог.,2019-01-01,1


Data Union

In [17]:
step1 = pd.merge(left=data, right=clients, left_on='client_inn', right_on='inn')
step1 = step1.drop(['inn'], axis=1)
step1.head()

,date_key,amount,client_inn,partner_inn,years,Длина_Отн_в_днях,Клиент_Сегмент_КБ,Клиент_Сегмент_МБ,Клиент_Сегмент_МКП,Клиент_Сегмент_СБ,Клиент_НаличиеЗПпроекта_Д,Клиент_НаличиеЗПпроекта_Н
0,2018-03-01,4217742.30,976651,976651,51.0,2103.0,0,0,0,1,1,0
1,2018-03-01,3621.48,976651,991728,51.0,2103.0,0,0,0,1,1,0
2,2018-03-01,2128.78,976651,1543616,51.0,2103.0,0,0,0,1,1,0
3,2018-03-01,6125.50,976651,1543616,51.0,2103.0,0,0,0,1,1,0
4,2018-03-01,2128.78,976651,1543616,51.0,2103.0,0,0,0,1,1,0


In [18]:
step2 = pd.merge_ordered(left=step1, right=fraud, left_on='client_inn', right_on='inn', fill_method=0)
step2 = step2.drop(['inn', 'month_fraud'], axis=1)
step2['Fraud_presence'] = step2['Fraud_presence'].fillna(0)
step2.head()

,date_key,amount,client_inn,partner_inn,years,Длина_Отн_в_днях,Клиент_Сегмент_КБ,Клиент_Сегмент_МБ,Клиент_Сегмент_МКП,Клиент_Сегмент_СБ,Клиент_НаличиеЗПпроекта_Д,Клиент_НаличиеЗПпроекта_Н,Fraud_presence
0,2018-03-01,13500.00,129,1910650.0,44.0,5479.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0
1,2018-03-01,8.00,129,119556.0,44.0,5479.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0
2,2018-03-01,4000.00,129,496375.0,44.0,5479.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0
3,2018-03-01,1453597.61,129,569265.0,44.0,5479.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0
4,2018-03-01,690.00,129,1382934.0,44.0,5479.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0


In [19]:
step3 = pd.merge_ordered(left=step2, right=reject, left_on='client_inn', right_on='inn', fill_method=0)
step3 = step3.drop(['inn', 'date_key', 'РеестрОтказов_ТипУчастника', 'РеестрОтказов_КодОтказаСтрокой', 'month_reject'], axis=1)
step3['Reject_presence'] = step3['Reject_presence'].fillna(0)
final_data = step3.dropna()
final_data.head()

,amount,client_inn,partner_inn,years,Длина_Отн_в_днях,Клиент_Сегмент_КБ,Клиент_Сегмент_МБ,Клиент_Сегмент_МКП,Клиент_Сегмент_СБ,Клиент_НаличиеЗПпроекта_Д,Клиент_НаличиеЗПпроекта_Н,Fraud_presence,Reject_presence
38,13500.00,129,1910650.0,44.0,5479.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
39,8.00,129,119556.0,44.0,5479.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
40,4000.00,129,496375.0,44.0,5479.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
41,1453597.61,129,569265.0,44.0,5479.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
42,690.00,129,1382934.0,44.0,5479.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0


In [ ]:
final_data.to_csv('data_for_save.csv') 

In [2]:
import os.path as osp

import torch
from torch_geometric.data import Dataset, Data

In [ ]:
#Такой перевод данных из .scv в датасет для графа показался мне самым правильным. Здесь сделано для молекул, но я думала нужно сделать тоже самое
#для моих данных

#Взято: https://github.com/deepfindr/gnn-project/blob/main/dataset.py

class MoleculeDataset(Dataset):
    def __init__(self, root, filename, test=False, transform=None, pre_transform=None):
        """
        root = Where the dataset should be stored. This folder is split
        into raw_dir (downloaded dataset) and processed_dir (processed data). 
        """
        self.test = test
        self.filename = filename
        super(MoleculeDataset, self).__init__(root, transform, pre_transform)
        
    @property
    def raw_file_names(self):
        """ If this file exists in raw_dir, the download is not triggered.
            (The download func. is not implemented here)  
        """
        return self.filename

    @property
    def processed_file_names(self):
        """ If these files are found in raw_dir, processing is skipped"""
        self.data = pd.read_csv(self.raw_paths[0]).reset_index()

        if self.test:
            return [f'data_test_{i}.pt' for i in list(self.data.index)]
        else:
            return [f'data_{i}.pt' for i in list(self.data.index)]

    def download(self):
        pass

    def process(self):
        self.data = pd.read_csv(self.raw_paths[0])
        for index, mol in tqdm(self.data.iterrows(), total=self.data.shape[0]):
            mol_obj = Chem.MolFromSmiles(mol["smiles"])
            # Get node features
            node_feats = self._get_node_features(mol_obj)
            # Get edge features
            edge_feats = self._get_edge_features(mol_obj)
            # Get adjacency info
            edge_index = self._get_adjacency_info(mol_obj)
            # Get labels info
            label = self._get_labels(mol["HIV_active"])

            # Create data object
            data = Data(x=node_feats, 
                        edge_index=edge_index,
                        edge_attr=edge_feats,
                        y=label,
                        smiles=mol["smiles"]
                        ) 
            if self.test:
                torch.save(data, 
                    os.path.join(self.processed_dir, 
                                 f'data_test_{index}.pt'))
            else:
                torch.save(data, 
                    os.path.join(self.processed_dir, 
                                 f'data_{index}.pt'))

    def _get_node_features(self, mol):
        """ 
        This will return a matrix / 2d array of the shape
        [Number of Nodes, Node Feature size]
        """
        all_node_feats = []

        for atom in mol.GetAtoms():
            node_feats = []
            # Feature 1: Atomic number        
            node_feats.append(atom.GetAtomicNum())
            # Feature 2: Atom degree
            node_feats.append(atom.GetDegree())
            # Feature 3: Formal charge
            node_feats.append(atom.GetFormalCharge())
            # Feature 4: Hybridization
            node_feats.append(atom.GetHybridization())
            # Feature 5: Aromaticity
            node_feats.append(atom.GetIsAromatic())
            # Feature 6: Total Num Hs
            node_feats.append(atom.GetTotalNumHs())
            # Feature 7: Radical Electrons
            node_feats.append(atom.GetNumRadicalElectrons())
            # Feature 8: In Ring
            node_feats.append(atom.IsInRing())
            # Feature 9: Chirality
            node_feats.append(atom.GetChiralTag())

            # Append node features to matrix
            all_node_feats.append(node_feats)

        all_node_feats = np.asarray(all_node_feats)
        return torch.tensor(all_node_feats, dtype=torch.float)

    def _get_edge_features(self, mol):
        """ 
        This will return a matrix / 2d array of the shape
        [Number of edges, Edge Feature size]
        """
        all_edge_feats = []

        for bond in mol.GetBonds():
            edge_feats = []
            # Feature 1: Bond type (as double)
            edge_feats.append(bond.GetBondTypeAsDouble())
            # Feature 2: Rings
            edge_feats.append(bond.IsInRing())
            # Append node features to matrix (twice, per direction)
            all_edge_feats += [edge_feats, edge_feats]

        all_edge_feats = np.asarray(all_edge_feats)
        return torch.tensor(all_edge_feats, dtype=torch.float)

    def _get_adjacency_info(self, mol):
        """
        We could also use rdmolops.GetAdjacencyMatrix(mol)
        but we want to be sure that the order of the indices
        matches the order of the edge features
        """
        edge_indices = []
        for bond in mol.GetBonds():
            i = bond.GetBeginAtomIdx()
            j = bond.GetEndAtomIdx()
            edge_indices += [[i, j], [j, i]]

        edge_indices = torch.tensor(edge_indices)
        edge_indices = edge_indices.t().to(torch.long).view(2, -1)
        return edge_indices

    def _get_labels(self, label):
        label = np.asarray([label])
        return torch.tensor(label, dtype=torch.int64)

    def len(self):
        return self.data.shape[0]

    def get(self, idx):
        """ - Equivalent to __getitem__ in pytorch
            - Is not needed for PyG's InMemoryDataset
        """
        if self.test:
            data = torch.load(os.path.join(self.processed_dir, 
                                 f'data_test_{idx}.pt'))
        else:
            data = torch.load(os.path.join(self.processed_dir, 
                                 f'data_{idx}.pt'))   
        return data

In [8]:
dataset = MyOwnDataset(root="data/")

AttributeError: 'MyOwnDataset' object has no attribute 'test'

In [ ]:
print(dataset[0].y)

In [ ]:
#Если говорить о простом переводе pandas в tensor, то правильно ли сделать так:

import torch

target_torch_tensor = torch.tensor(final_data['Fraud_presence'].values)

#Но мне не понятно, почему мы используем только отклики, и не используем все данные.
#И где в данном тензоне будет представлены данные, в которых содержится информация об узлах и ребрах?

In [19]:
#Это код с гитхаба, который вы мне скинули, где сворачивают признаки по соседям и
#получают агрегированные признаки вершин

import argparse
from tqdm import tqdm

import torch
import torch.nn.functional as F
from torch_sparse import SparseTensor
from torch_geometric.utils import to_undirected, dropout_adj
import os
import os.path as osp

from ogb.nodeproppred import PygNodePropPredDataset

In [24]:
import torch
from torch_geometric.data import Data

In [25]:
data = Data()

In [23]:
train6 = pd.concat([Year_2018_P1, Year_2018_P2, Year_2019_P1, Year_2019_P2, Year_2020_P1, Year_2020_P2], ignore_index=True)
train6 = train6.drop(['date_key'], axis=1)
train_data6 = train6.drop(['Fraud_presence', 'Reject_presence'], axis=1)
train_labels6 = train6['Fraud_presence']
test_data6 = Year_2021.drop(['Fraud_presence', 'Reject_presence', 'date_key'], axis=1)
test_labels6 = Year_2021['Fraud_presence']

In [22]:
parser = argparse.ArgumentParser()
parser.add_argument('--num_hops', type=int, default=6)
parser.add_argument('--root', type=str, default='./')
args = parser.parse_args()
print(args)

dataset = PygNodePropPredDataset('ogbn-papers100M', root=args.root)
split_idx = dataset.get_idx_split()
train_idx, valid_idx, test_idx = split_idx['train'], split_idx['valid'], split_idx['test']
data = dataset[0]
x = data.x
N = data.num_nodes

usage: ipykernel_launcher.py [-h] [--num_hops NUM_HOPS] [--root ROOT]
ipykernel_launcher.py: error: unrecognized arguments: -f /data/home/krazheva/.local/share/jupyter/runtime/kernel-e404630f-1176-4847-b72a-090e8ac1e113.json


SystemExit: 2

/data/home/krazheva/.conda/envs/py3.10/lib/python3.10/site-packages/IPython/core/interactiveshell.py:3406: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


In [20]:


path = './adj_gcn.pt'
print('Making the graph undirected.')

data.edge_index, _ = dropout_adj(
       data.edge_index, p=0, num_nodes=data.num_nodes)
data.edge_index = to_undirected(data.edge_index, num_nodes=data.num_nodes)
print(data)

row, col = data.edge_index
print('Computing adj...')

adj = SparseTensor(row=row, col=col, sparse_sizes=(N, N))
adj = adj.set_diag()
deg = adj.sum(dim=1).to(torch.float)
deg_inv_sqrt = deg.pow(-0.5)
deg_inv_sqrt[deg_inv_sqrt == float('inf')] = 0
adj = deg_inv_sqrt.view(-1, 1) * adj * deg_inv_sqrt.view(1, -1)
adj = adj.to_scipy(layout='csr')

def sparse_mx_to_torch_sparse_tensor(sparse_mx):
    """Convert a scipy sparse matrix to a torch sparse tensor."""
    sparse_mx = sparse_mx.tocoo().astype(np.float32)
    indices = torch.from_numpy(np.vstack((sparse_mx.row, sparse_mx.col)).astype(np.int64))
    values = torch.from_numpy(sparse_mx.data)
    shape = torch.Size(sparse_mx.shape)
    return torch.sparse.FloatTensor(indices, values, shape)

adj=sparse_mx_to_torch_sparse_tensor(adj)
print('Start processing')
saved = torch.cat((x[train_idx], x[valid_idx], x[test_idx]), dim=0)
torch.save(saved, f'./papers100m_feat_0.pt')

for i in tqdm(range(args.num_hops)):
    x = adj @ x
    saved = torch.cat((x[train_idx], x[valid_idx], x[test_idx]), dim=0)
    torch.save(saved, f'./papers100m_feat_{i+1}.pt')

NameError: name 'args' is not defined